In [1]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
# using CuArrays

In [2]:
# Classify MNIST digits with a simple multi-layer-perceptron
imgs = Flux.Data.MNIST.images()
labels = Flux.Data.MNIST.labels();

┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/jismysebastian/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    200      0  0:00:02  0:00:02 --:--:--   200
100 9680k  100 9680k    0     0   305k      0  0:00:31  0:00:31 --:--:--  490k
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/jismysebastian/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    283      0  0:00:01  0:00:01 --:--:--   283
100 28881  100 28881    0     0   7793      0  0:00:03  0:00:03 --:--:-- 16550
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /home/jismysebastian/.julia/packages/Flux/qXNjB/src/data/mnist.jl:24

In [4]:
# Stack images into one large batch
X = hcat(float.(reshape.(imgs, :))...) |> gpu

784×60000 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  

In [5]:
labels = MNIST.labels()
# One-hot-encode the labels
Y = onehotbatch(labels, 0:9) |> gpu


10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 false   true  false  false  false  …  false  false  false  false  false
 false  false  false   true  false     false  false  false  false  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false     false   true  false  false  false
 false  false   true  false  false     false  false  false  false  false
  true  false  false  false  false  …  false  false   true  false  false
 false  false  false  false  false     false  false  false   true  false
 false  false  false  false  false     false  false  false  false  false
 false  false  false  false  false      true  false  false  false   true
 false  false  false  false   true     false  false  false  false  false

In [6]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)


loss (generic function with 1 method)

In [18]:
imgs[6021]

In [19]:
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [20]:
dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM(params(m))

Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 10))

println("acc X,Y ", accuracy(X, Y))

┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at In[20]:3
└ @ Core In[20]:3
┌ Warning: train!(loss, data, opt) is deprecated; use train!(loss, params, data, opt) instead
│   caller = ip:0x0
└ @ Core :-1


loss(X, Y) = 2.2717795f0 (tracked)
loss(X, Y) = 2.2717795f0 (tracked)
loss(X, Y) = 2.2717795f0 (tracked)
loss(X, Y) = 2.2717795f0 (tracked)
loss(X, Y) = 2.2717795f0 (tracked)
loss(X, Y) = 2.2717795f0 (tracked)
loss(X, Y) = 2.2717795f0 (tracked)
acc X,Y 0.10953333333333333


In [21]:

# Test set accuracy
tX = hcat(float.(reshape.(MNIST.images(:test), :))...) |> gpu
tY = onehotbatch(MNIST.labels(:test), 0:9) |> gpu

println("acc tX, tY ", accuracy(tX, tY))


acc tX, tY 0.1092


#### more sophisticated model which uses a convolutional Neural Network. 

In [23]:
using Base.Iterators: repeated, partition

labels = onehotbatch(MNIST.labels(), 0:9)

# Partition into batches of size 1,000
train = [(cat(float.(imgs[i])..., dims = 4), labels[:,i])
         for i in partition(1:60_000, 1000)]

train = gpu.(train)

60-element Array{Tuple{Array{Float64,4},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}},1}:
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [false true … false false; false false … false false; … ; false false … false false; false false … false false]) 
 ([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 

In [24]:
# Prepare test set (first 1,000 images)
tX = cat(float.(MNIST.images(:test)[1:1000])..., dims = 4) |> gpu
tY = onehotbatch(MNIST.labels(:test)[1:1000], 0:9) |> gpu

m = Chain(
  Conv((2,2), 1=>16, relu),
  x -> maxpool(x, (2,2)),
  Conv((2,2), 16=>8, relu),
  x -> maxpool(x, (2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(288, 10), softmax) |> gpu


Chain(Conv((2, 2), 1=>16, NNlib.relu), getfield(Main, Symbol("##9#12"))(), Conv((2, 2), 16=>8, NNlib.relu), getfield(Main, Symbol("##10#13"))(), getfield(Main, Symbol("##11#14"))(), Dense(288, 10), NNlib.softmax)

In [25]:
m(train[1][1])

MethodError: MethodError: no method matching maxpool(::TrackedArray{…,Array{Float64,4}}, ::Tuple{Int64,Int64})
Closest candidates are:
  maxpool(::TrackedArray, !Matched::PoolDims; kw...) at /home/jismysebastian/.julia/packages/Tracker/RRYy6/src/lib/array.jl:446
  maxpool(::AbstractArray{xT,N}, !Matched::PoolDims; kwargs...) where {xT, N} at /home/jismysebastian/.julia/packages/TimerOutputs/7zSea/src/TimerOutput.jl:198

In [26]:
loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

evalcb = throttle(() -> @show(accuracy(tX, tY)), 10)
opt = ADAM(params(m))

Flux.train!(loss, train, opt, cb = evalcb)


┌ Warning: ADAM(params) is deprecated; use ADAM(η::Float64) instead
│   caller = top-level scope at In[26]:6
└ @ Core In[26]:6


MethodError: MethodError: no method matching maxpool(::TrackedArray{…,Array{Float64,4}}, ::Tuple{Int64,Int64})
Closest candidates are:
  maxpool(::TrackedArray, !Matched::PoolDims; kw...) at /home/jismysebastian/.julia/packages/Tracker/RRYy6/src/lib/array.jl:446
  maxpool(::AbstractArray{xT,N}, !Matched::PoolDims; kwargs...) where {xT, N} at /home/jismysebastian/.julia/packages/TimerOutputs/7zSea/src/TimerOutput.jl:198